# Neo4j Test

In [1]:
!pip install neo4j

In [3]:
from neo4j import GraphDatabase
import pandas as pd

In [8]:
url = 'https://raw.githubusercontent.com/ngshya/datasets/master/cora/cora_content.csv'
papers = pd.read_csv(url, delimiter = ',')
papers.head(10)

,paper_id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1425,feature_1426,feature_1427,feature_1428,feature_1429,feature_1430,feature_1431,feature_1432,feature_1433,label
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods
5,1126012,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Probabilistic_Methods
6,1107140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Theory
7,1102850,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neural_Networks
8,31349,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neural_Networks
9,1106418,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Theory


In [9]:
query_string = '''
    USING PERIODIC COMMIT 500
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/ngshya/datasets/master/cora/cora_content.csv"
    AS line FIELDTERMINATOR ','
    CREATE (:Paper {id: line.paper_id, class: line.label}) 
    ''' 

In [11]:
  # Create a connection class
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)      
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()  

In [12]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="cora")

In [15]:
conn.query("CREATE OR REPLACE DATABASE neo4j")

[]

In [16]:
conn.query(query_string, db='neo4j')

[]

In [18]:
url = 'https://raw.githubusercontent.com/ngshya/datasets/master/cora/cora_cites.csv'
cites = pd.read_csv(url, delimiter=',')
cites.head(10)

,cited_paper_id,citing_paper_id
0,35,1033
1,35,103482
2,35,103515
3,35,1050679
4,35,1103960
5,35,1103985
6,35,1109199
7,35,1112911
8,35,1113438
9,35,1113831


In [19]:
query_string = '''
    USING PERIODIC COMMIT 500
    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ngshya/datasets/master/cora/cora_cites.csv'
    AS line FIELDTERMINATOR ','
    MATCH (citing_paper:Paper {id: line.citing_paper_id}),(cited_paper:Paper {id: line.cited_paper_id})
    CREATE (citing_paper)-[:CITES]->(cited_paper)
    '''

In [26]:
conn.query(query_string, db='neo4j')

Query failed: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `gds.graph.create` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}


In [27]:
query_string = '''
    CALL gds.graph.create(
        'Highways',
        'Place',
        {
            EROAD: {
                orientation: 'UNDIRECTED'
            }
        }
    )
    '''

In [28]:
conn.query(query_string, db='neo4j')

Query failed: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `gds.graph.create` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}


In [33]:
query_string = '''
    CALL gds.triangleCount.stream('Paper')
    YIELD nodeId, triangleCount
    RETURN gds.util.asNode(nodeId).id
    AS name, triangleCount
    ORDER BY triangleCount DESC
    '''

In [34]:
conn.query(query_string, db='neo4j')

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.triangleCount.stream`: Caused by: java.util.NoSuchElementException: Graph with name `Paper` does not exist on database `neo4j`. It might exist on another database.}
